In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sthrd_benchmark import (
    DynamicEnvironment,
    BenchmarkRunner,
    FirstOrderOracle,
    ZeroOrderOracle,
    HybridOracle,
    Drift,
    StationaryDrift,
    LinearDrift,
    RandomWalkDrift,
    CyclicDrift,
    TeleportationDrift,
    Landscape,
    QuadraticLandscape,
    QuadraticRavineLandscape,
    PNormLandscape,
    RosenbrockLandscape,
    Noise,
    GaussianNoise,
    ParetoNoise,
    BernoulliSparseNoise,
    QuantizationNoise,
    Metric,
    TrackingErrorMetric,
    DynamicRegretMetric,
    TimeToRecoveryMetric,
    make_default_environment,
    make_benchmark,
)

from sthrd_benchmark.visualization import BenchmarkVisualizer

print("=" * 80)
print("ST-HRD BENCHMARK - COMPREHENSIVE DEMONSTRATION")
print("=" * 80)

print("\n" + "=" * 80)
print("1. BASIC SETUP - Simple SGD with Random Walk Drift")
print("=" * 80)


class SimpleSGD:
    def __init__(self, dim, lr=0.01, momentum=0.9):
        self.dim = dim
        self.lr = lr
        self.momentum = momentum
        self.velocity = np.zeros(dim)

    def step(self, x, oracle_data):
        grad = oracle_data["grad"]
        self.velocity = self.momentum * self.velocity + self.lr * grad
        return x - self.velocity


drift = RandomWalkDrift(dim=5, step_size=0.05, seed=42)
landscape = QuadraticLandscape(dim=5, condition_number=10.0, seed=42)
env = DynamicEnvironment(drift=drift, landscape=landscape, seed=42)

oracle = FirstOrderOracle(
    environment=env,
    grad_noise=GaussianNoise(std=0.1, seed=42),
    value_noise=GaussianNoise(std=0.01, seed=42),
)

metrics = [
    TrackingErrorMetric(normalize_by_dim=True, name="NormalizedError"),
    TrackingErrorMetric(norm=1, name="L1Error"),
    DynamicRegretMetric(name="DynamicRegret"),
]

runner = BenchmarkRunner(
    environment=env,
    oracle=oracle,
    metrics=metrics,
    config={"n_steps": 100, "eval_frequency": 5},
)

sgd = SimpleSGD(dim=5, lr=0.01)
results_sgd = runner.run(sgd, verbose=True)

print("\nSimple SGD Results:")
print(f"Final tracking error: {results_sgd['final_state']['final_error']:.4f}")
print("Metrics:")
for name, value in results_sgd["metrics"].items():
    if value is not None:
        print(f"  {name}: {value:.4f}")

viz_sgd = BenchmarkVisualizer(results_sgd)
print("\nGenerating visualizations for Simple SGD...")

fig = make_subplots(
    rows=2,
    cols=2,
    subplot_titles=(
        "Tracking Error Convergence",
        "2D Trajectory",
        "Metrics Dashboard",
        "State Evolution",
    ),
    specs=[
        [{"type": "scatter"}, {"type": "scatter"}],
        [{"type": "bar"}, {"type": "scatter"}],
    ],
)

history = results_sgd["history"]
errors = [np.linalg.norm(x - theta) for x, theta in zip(history["x"], history["theta"])]
fig.add_trace(
    go.Scatter(
        x=history["time"],
        y=errors,
        mode="lines",
        name="Error",
        line=dict(color="red", width=2),
    ),
    row=1,
    col=1,
)

if len(history["x"]) > 0 and len(history["x"][0]) >= 2:
    x_vals = [x[0] for x in history["x"]]
    y_vals = [x[1] for x in history["x"]]
    theta_x = [t[0] for t in history["theta"]]
    theta_y = [t[1] for t in history["theta"]]

    fig.add_trace(
        go.Scatter(
            x=x_vals,
            y=y_vals,
            mode="lines",
            name="Algorithm",
            line=dict(color="blue", width=3),
        ),
        row=1,
        col=2,
    )
    fig.add_trace(
        go.Scatter(
            x=theta_x,
            y=theta_y,
            mode="lines",
            name="Target",
            line=dict(color="red", width=3, dash="dash"),
        ),
        row=1,
        col=2,
    )

metrics_data = results_sgd["metrics"]
metric_names = [k for k in metrics_data.keys() if metrics_data[k] is not None]
metric_values = [metrics_data[k] for k in metric_names]
fig.add_trace(go.Bar(x=metric_names, y=metric_values, name="Metrics"), row=2, col=1)

if len(history["x"]) > 0:
    x0 = [x[0] for x in history["x"]]
    theta0 = [t[0] for t in history["theta"]]
    fig.add_trace(
        go.Scatter(
            x=history["time"],
            y=x0,
            mode="lines",
            name="x₀",
            line=dict(color="blue", width=2),
        ),
        row=2,
        col=2,
    )
    fig.add_trace(
        go.Scatter(
            x=history["time"],
            y=theta0,
            mode="lines",
            name="θ₀",
            line=dict(color="red", width=2, dash="dash"),
        ),
        row=2,
        col=2,
    )

fig.update_layout(height=800, width=1000, title_text="Simple SGD Analysis")
fig.show()

print("\n" + "=" * 80)
print("2. COMPARING DIFFERENT OPTIMIZERS")
print("=" * 80)


class MomentumSGD:
    def __init__(self, dim, lr=0.01, momentum=0.9):
        self.dim = dim
        self.lr = lr
        self.momentum = momentum
        self.velocity = np.zeros(dim)

    def step(self, x, oracle_data):
        grad = oracle_data["grad"]
        self.velocity = self.momentum * self.velocity + grad
        return x - self.lr * self.velocity


class AdamOptimizer:
    def __init__(self, dim, lr=0.01, beta1=0.9, beta2=0.999, eps=1e-8):
        self.dim = dim
        self.lr = lr
        self.beta1 = beta1
        self.beta2 = beta2
        self.eps = eps
        self.m = np.zeros(dim)
        self.v = np.zeros(dim)
        self.t = 0

    def step(self, x, oracle_data):
        grad = oracle_data["grad"]
        self.t += 1

        self.m = self.beta1 * self.m + (1 - self.beta1) * grad
        self.v = self.beta2 * self.v + (1 - self.beta2) * (grad**2)

        m_hat = self.m / (1 - self.beta1**self.t)
        v_hat = self.v / (1 - self.beta2**self.t)

        update = self.lr * m_hat / (np.sqrt(v_hat) + self.eps)
        return x - update


optimizers = {
    "SGD (lr=0.01)": SimpleSGD(dim=5, lr=0.01),
    "SGD (lr=0.05)": SimpleSGD(dim=5, lr=0.05),
    "Momentum SGD": MomentumSGD(dim=5, lr=0.01, momentum=0.9),
    "Adam": AdamOptimizer(dim=5, lr=0.1),
}

all_results = {}
for name, optimizer in optimizers.items():
    print(f"\nRunning {name}...")
    env.reset()
    runner = BenchmarkRunner(
        environment=env,
        oracle=oracle,
        metrics=[TrackingErrorMetric(name="Error")],
        config={"n_steps": 50, "eval_frequency": 5},
    )
    results = runner.run(optimizer, verbose=False)
    all_results[name] = results
    print(f"  Final error: {results['final_state']['final_error']:.4f}")

fig = go.Figure()

for name, results in all_results.items():
    history = results["history"]
    errors = [
        np.linalg.norm(x - theta) for x, theta in zip(history["x"], history["theta"])
    ]
    fig.add_trace(go.Scatter(x=history["time"], y=errors, mode="lines", name=name))

fig.update_layout(
    title="Optimizer Comparison - Tracking Error",
    xaxis_title="Time Step",
    yaxis_title="Error",
    yaxis_type="log",
)
fig.show()

fig = go.Figure(
    data=[
        go.Bar(
            x=list(all_results.keys()),
            y=[
                results["final_state"]["final_error"]
                for results in all_results.values()
            ],
            text=[
                f"{results['final_state']['final_error']:.4f}"
                for results in all_results.values()
            ],
            textposition="auto",
        )
    ]
)
fig.update_layout(
    title="Final Tracking Error Comparison",
    xaxis_title="Optimizer",
    yaxis_title="Final Error",
)
fig.show()

print("\n" + "=" * 80)
print("3. TESTING DIFFERENT DRIFT TYPES")
print("=" * 80)

drift_types = {
    "Stationary": StationaryDrift(dim=3, seed=42),
    "Linear": LinearDrift(dim=3, step_size=0.03, seed=42),
    "Random Walk": RandomWalkDrift(dim=3, step_size=0.05, seed=42),
    "Cyclic": CyclicDrift(dim=3, period=20, amplitude=1.0, seed=42),
    "Teleportation": TeleportationDrift(
        dim=3, teleport_prob=0.02, jump_scale=2.0, seed=42
    ),
}

landscape = QuadraticLandscape(dim=3, condition_number=5.0, seed=42)

drift_results = {}
for drift_name, drift in drift_types.items():
    print(f"\nTesting {drift_name} drift...")
    env = DynamicEnvironment(drift=drift, landscape=landscape, seed=42)

    oracle = FirstOrderOracle(
        environment=env,
        grad_noise=GaussianNoise(std=0.05, seed=42),
        value_noise=GaussianNoise(std=0.005, seed=42),
    )

    runner = BenchmarkRunner(
        environment=env,
        oracle=oracle,
        metrics=[TrackingErrorMetric(name="Error")],
        config={"n_steps": 50, "eval_frequency": 5},
    )

    optimizer = SimpleSGD(dim=3, lr=0.02)
    results = runner.run(optimizer, verbose=False)
    drift_results[drift_name] = results
    print(f"  Final error: {results['final_state']['final_error']:.4f}")

fig = make_subplots(
    rows=2,
    cols=3,
    subplot_titles=list(drift_results.keys()),
    vertical_spacing=0.15,
    horizontal_spacing=0.1,
)

for i, (drift_name, results) in enumerate(drift_results.items()):
    row = i // 3 + 1
    col = i % 3 + 1

    history = results["history"]
    if len(history["x"]) > 0 and len(history["x"][0]) >= 2:
        x_vals = [x[0] for x in history["x"]]
        y_vals = [x[1] for x in history["x"]]
        theta_x = [t[0] for t in history["theta"]]
        theta_y = [t[1] for t in history["theta"]]

        fig.add_trace(
            go.Scatter(
                x=x_vals,
                y=y_vals,
                mode="lines",
                name=f"{drift_name} (Algo)",
                line=dict(width=2),
                showlegend=False,
            ),
            row=row,
            col=col,
        )
        fig.add_trace(
            go.Scatter(
                x=theta_x,
                y=theta_y,
                mode="lines",
                name=f"{drift_name} (Target)",
                line=dict(width=2, dash="dash"),
                showlegend=False,
            ),
            row=row,
            col=col,
        )

fig.update_layout(
    height=600, width=900, title_text="Different Drift Types - 2D Trajectories"
)
fig.show()

print("\n" + "=" * 80)
print("4. TESTING DIFFERENT LANDSCAPE TYPES")
print("=" * 80)

landscape_types = {
    "Quadratic (cond=5)": QuadraticLandscape(dim=3, condition_number=5.0, seed=42),
    "Quadratic (cond=50)": QuadraticLandscape(dim=3, condition_number=50.0, seed=42),
    "Quadratic Ravine": QuadraticRavineLandscape(dim=3, condition_number=20.0, seed=42),
    "p-Norm (p=3)": PNormLandscape(dim=3, p=3.0, seed=42),
    "Rosenbrock": RosenbrockLandscape(dim=3, seed=42),
}

drift = RandomWalkDrift(dim=3, step_size=0.03, seed=42)

landscape_results = {}
for landscape_name, landscape in landscape_types.items():
    print(f"\nTesting {landscape_name}...")
    env = DynamicEnvironment(drift=drift, landscape=landscape, seed=42)

    oracle = FirstOrderOracle(
        environment=env,
        grad_noise=GaussianNoise(std=0.1, seed=42),
        value_noise=GaussianNoise(std=0.01, seed=42),
    )

    runner = BenchmarkRunner(
        environment=env,
        oracle=oracle,
        metrics=[TrackingErrorMetric(name="Error")],
        config={"n_steps": 50, "eval_frequency": 5},
    )

    optimizer = SimpleSGD(dim=3, lr=0.01)
    results = runner.run(optimizer, verbose=False)
    landscape_results[landscape_name] = results
    print(f"  Final error: {results['final_state']['final_error']:.4f}")

fig = go.Figure()
for landscape_name, results in landscape_results.items():
    history = results["history"]
    errors = [
        np.linalg.norm(x - theta) for x, theta in zip(history["x"], history["theta"])
    ]
    fig.add_trace(
        go.Scatter(x=history["time"], y=errors, mode="lines", name=landscape_name)
    )

fig.update_layout(
    title="Different Landscapes - Error Convergence",
    xaxis_title="Time Step",
    yaxis_title="Tracking Error",
    yaxis_type="log",
)
fig.show()

print("\n" + "=" * 80)
print("5. TESTING DIFFERENT NOISE TYPES")
print("=" * 80)

noise_types = {
    "Gaussian (σ=0.1)": GaussianNoise(std=0.1, seed=42),
    "Gaussian (σ=0.5)": GaussianNoise(std=0.5, seed=42),
    "Pareto (α=2.5)": ParetoNoise(alpha=2.5, scale=0.1, seed=42),
    "Sparse (10%)": BernoulliSparseNoise(sparsity=0.1, scale=1.0, seed=42),
    "Quantization (10 levels)": QuantizationNoise(levels=10, scale=0.1, seed=42),
}

drift = RandomWalkDrift(dim=3, step_size=0.03, seed=42)
landscape = QuadraticLandscape(dim=3, condition_number=10.0, seed=42)

noise_results = {}
for noise_name, noise in noise_types.items():
    print(f"\nTesting {noise_name}...")
    env = DynamicEnvironment(drift=drift, landscape=landscape, seed=42)

    oracle = FirstOrderOracle(
        environment=env, grad_noise=noise, value_noise=GaussianNoise(std=0.01, seed=42)
    )

    runner = BenchmarkRunner(
        environment=env,
        oracle=oracle,
        metrics=[TrackingErrorMetric(name="Error")],
        config={"n_steps": 50, "eval_frequency": 5},
    )

    optimizer = SimpleSGD(dim=3, lr=0.01)
    results = runner.run(optimizer, verbose=False)
    noise_results[noise_name] = results
    print(f"  Final error: {results['final_state']['final_error']:.4f}")

fig = go.Figure()
for noise_name, results in noise_results.items():
    history = results["history"]
    errors = [
        np.linalg.norm(x - theta) for x, theta in zip(history["x"], history["theta"])
    ]
    fig.add_trace(
        go.Scatter(x=history["time"], y=errors, mode="lines", name=noise_name)
    )

fig.update_layout(
    title="Different Noise Types - Error Convergence",
    xaxis_title="Time Step",
    yaxis_title="Tracking Error",
)
fig.show()

print("\n" + "=" * 80)
print("6. TESTING DIFFERENT ORACLE TYPES")
print("=" * 80)

drift = RandomWalkDrift(dim=3, step_size=0.02, seed=42)
landscape = QuadraticLandscape(dim=3, condition_number=5.0, seed=42)
env = DynamicEnvironment(drift=drift, landscape=landscape, seed=42)

oracle_types = {
    "First-Order (Gaussian)": FirstOrderOracle(
        environment=env,
        grad_noise=GaussianNoise(std=0.1, seed=42),
        value_noise=GaussianNoise(std=0.01, seed=42),
    ),
    "Zero-Order": ZeroOrderOracle(
        environment=env,
        perturbation_scale=0.05,
        value_noise=GaussianNoise(std=0.01, seed=42),
        seed=42,
    ),
    "Hybrid (10% sparse)": HybridOracle(
        environment=env,
        grad_noise=GaussianNoise(std=0.1, seed=42),
        value_noise=GaussianNoise(std=0.01, seed=42),
        sparse_prob=0.1,
        seed=42,
    ),
}

oracle_results = {}
for oracle_name, oracle_obj in oracle_types.items():
    print(f"\nTesting {oracle_name}...")

    runner = BenchmarkRunner(
        environment=env,
        oracle=oracle_obj,
        metrics=[TrackingErrorMetric(name="Error")],
        config={"n_steps": 50, "eval_frequency": 5},
    )

    if "Zero" in oracle_name:
        lr = 0.05
    elif "Hybrid" in oracle_name:
        lr = 0.02
    else:
        lr = 0.01

    optimizer = SimpleSGD(dim=3, lr=lr)
    results = runner.run(optimizer, verbose=False)
    oracle_results[oracle_name] = results
    print(f"  Learning rate: {lr}")
    print(f"  Final error: {results['final_state']['final_error']:.4f}")
    print(f"  Oracle calls: {results['history']['oracle_calls']}")

fig = make_subplots(
    rows=1,
    cols=2,
    subplot_titles=("Final Error", "Oracle Calls"),
    specs=[[{"type": "bar"}, {"type": "bar"}]],
)

errors = [results["final_state"]["final_error"] for results in oracle_results.values()]
fig.add_trace(
    go.Bar(
        x=list(oracle_results.keys()),
        y=errors,
        name="Final Error",
        text=[f"{e:.4f}" for e in errors],
        textposition="auto",
    ),
    row=1,
    col=1,
)

calls = [results["history"]["oracle_calls"] for results in oracle_results.values()]
fig.add_trace(
    go.Bar(
        x=list(oracle_results.keys()),
        y=calls,
        name="Oracle Calls",
        text=[f"{c}" for c in calls],
        textposition="auto",
    ),
    row=1,
    col=2,
)

fig.update_layout(height=500, width=900, title_text="Oracle Type Comparison")
fig.show()

fig = go.Figure()
for oracle_name, results in oracle_results.items():
    history = results["history"]
    errors = [
        np.linalg.norm(x - theta) for x, theta in zip(history["x"], history["theta"])
    ]
    fig.add_trace(
        go.Scatter(x=history["time"], y=errors, mode="lines", name=oracle_name)
    )

fig.update_layout(
    title="Oracle Types - Error Convergence",
    xaxis_title="Time Step",
    yaxis_title="Tracking Error",
)
fig.show()

print("\n" + "=" * 80)
print("7. USING make_benchmark CONVENIENCE FUNCTION")
print("=" * 80)

print("\nCreating benchmark with default settings...")
benchmark = make_benchmark(dim=5, n_steps=50, noise_scale=0.1, seed=42)

print("\nRunning with Simple SGD...")
results = benchmark.run(SimpleSGD(dim=5, lr=0.01), verbose=True)

print("\nResults:")
print(f"Final error: {results['final_state']['final_error']:.4f}")
print("Metrics:")
for name, value in results["metrics"].items():
    if value is not None:
        print(f"  {name}: {value:.4f}")

df = benchmark.get_dataframe()
print(f"\nDataFrame shape: {df.shape}")
print("\nFirst few rows:")
print(df.head())

fig = make_subplots(
    rows=2,
    cols=2,
    subplot_titles=(
        "Loss over Time",
        "Error over Time",
        "State Norms",
        "First Dimension",
    ),
)

fig.add_trace(
    go.Scatter(x=df["time"], y=df["loss"], mode="lines", name="Loss"), row=1, col=1
)

fig.add_trace(
    go.Scatter(x=df["time"], y=df["error"], mode="lines", name="Error"), row=1, col=2
)

fig.add_trace(
    go.Scatter(x=df["time"], y=df["x_norm"], mode="lines", name="|x|"), row=2, col=1
)
fig.add_trace(
    go.Scatter(x=df["time"], y=df["theta_norm"], mode="lines", name="|θ|"), row=2, col=1
)

if "x_0" in df.columns:
    fig.add_trace(
        go.Scatter(x=df["time"], y=df["x_0"], mode="lines", name="x₀"), row=2, col=2
    )
    fig.add_trace(
        go.Scatter(x=df["time"], y=df["theta_0"], mode="lines", name="θ₀"), row=2, col=2
    )

fig.update_layout(height=700, width=900, title_text="make_benchmark Results Analysis")
fig.show()

print("\n" + "=" * 80)
print("8. LEARNING RATE TUNING EXPERIMENT")
print("=" * 80)

print("\nSetting up learning rate tuning experiment...")


def run_lr_sweep(env, oracle, lr_values, n_trials=3, n_steps=30):
    results = []

    for lr in lr_values:
        trial_errors = []
        for trial in range(n_trials):
            env.reset()

            runner = BenchmarkRunner(
                environment=env,
                oracle=oracle,
                metrics=[TrackingErrorMetric(name="Error")],
                config={"n_steps": n_steps, "eval_frequency": 5},
            )

            optimizer = SimpleSGD(dim=env.dim, lr=lr)
            result = runner.run(optimizer, verbose=False)
            trial_errors.append(result["final_state"]["final_error"])

        avg_error = np.mean(trial_errors)
        std_error = np.std(trial_errors)
        results.append(
            {
                "lr": lr,
                "avg_error": avg_error,
                "std_error": std_error,
                "trial_errors": trial_errors,
            }
        )
        print(f"  lr={lr:.4f}: avg error={avg_error:.4f} ± {std_error:.4f}")

    return results


print("\nRunning learning rate sweep...")
drift = RandomWalkDrift(dim=3, step_size=0.03, seed=42)
landscape = QuadraticLandscape(dim=3, condition_number=10.0, seed=42)
env = DynamicEnvironment(drift=drift, landscape=landscape, seed=42)

oracle = FirstOrderOracle(
    environment=env,
    grad_noise=GaussianNoise(std=0.1, seed=42),
    value_noise=GaussianNoise(std=0.01, seed=42),
)

lr_sweep_results = run_lr_sweep(
    env=env,
    oracle=oracle,
    lr_values=[0.001, 0.005, 0.01, 0.02, 0.05, 0.1, 0.2],
    n_trials=3,
    n_steps=30,
)

fig = go.Figure()

lrs = [r["lr"] for r in lr_sweep_results]
avg_errors = [r["avg_error"] for r in lr_sweep_results]
std_errors = [r["std_error"] for r in lr_sweep_results]

fig.add_trace(
    go.Scatter(
        x=lrs,
        y=avg_errors,
        error_y=dict(type="data", array=std_errors, visible=True),
        mode="lines+markers",
        name="Average Error",
    )
)

best_idx = np.argmin(avg_errors)
best_lr = lrs[best_idx]
best_error = avg_errors[best_idx]

fig.add_trace(
    go.Scatter(
        x=[best_lr],
        y=[best_error],
        mode="markers",
        marker=dict(size=15, color="red", symbol="star"),
        name=f"Best LR: {best_lr:.4f}",
    )
)

fig.update_layout(
    title=f"Learning Rate Tuning (Best: LR={best_lr:.4f}, Error={best_error:.4f})",
    xaxis_title="Learning Rate",
    yaxis_title="Final Error",
    xaxis_type="log",
    yaxis_type="log",
)

fig.show()

print("\n" + "=" * 80)
print("9. COMPREHENSIVE RADAR CHART COMPARISON")
print("=" * 80)

configs = {
    "SGD + Gaussian": {
        "optimizer": SimpleSGD(dim=3, lr=0.01),
        "drift": RandomWalkDrift(dim=3, step_size=0.03, seed=42),
        "landscape": QuadraticLandscape(dim=3, condition_number=5.0, seed=42),
        "noise": GaussianNoise(std=0.1, seed=42),
    },
    "SGD + Pareto": {
        "optimizer": SimpleSGD(dim=3, lr=0.01),
        "drift": RandomWalkDrift(dim=3, step_size=0.03, seed=42),
        "landscape": QuadraticLandscape(dim=3, condition_number=5.0, seed=42),
        "noise": ParetoNoise(alpha=2.5, scale=0.1, seed=42),
    },
    "Adam + Linear Drift": {
        "optimizer": AdamOptimizer(dim=3, lr=0.1),
        "drift": LinearDrift(dim=3, step_size=0.03, seed=42),
        "landscape": QuadraticLandscape(dim=3, condition_number=5.0, seed=42),
        "noise": GaussianNoise(std=0.1, seed=42),
    },
    "Momentum + Ravine": {
        "optimizer": MomentumSGD(dim=3, lr=0.01, momentum=0.9),
        "drift": RandomWalkDrift(dim=3, step_size=0.03, seed=42),
        "landscape": QuadraticRavineLandscape(dim=3, condition_number=20.0, seed=42),
        "noise": GaussianNoise(std=0.1, seed=42),
    },
}

comparison_metrics = {}
for config_name, config in configs.items():
    print(f"\nTesting {config_name}...")

    env = DynamicEnvironment(
        drift=config["drift"], landscape=config["landscape"], seed=42
    )

    oracle = FirstOrderOracle(
        environment=env,
        grad_noise=config["noise"],
        value_noise=GaussianNoise(std=0.01, seed=42),
    )

    metrics = [
        TrackingErrorMetric(name="FinalError"),
        TrackingErrorMetric(norm=1, name="L1Error"),
        DynamicRegretMetric(name="Regret"),
    ]

    runner = BenchmarkRunner(
        environment=env,
        oracle=oracle,
        metrics=metrics,
        config={"n_steps": 40, "eval_frequency": 5},
    )

    results = runner.run(config["optimizer"], verbose=False)

    comparison_metrics[config_name] = {
        "FinalError": results["final_state"]["final_error"],
        "L1Error": results["metrics"].get("L1Error", 0) or 0,
        "Regret": results["metrics"].get("Regret", 0) or 0,
        "OracleCalls": results["history"]["oracle_calls"],
    }

    print(f"  Final error: {results['final_state']['final_error']:.4f}")

metrics_list = ["FinalError", "L1Error", "Regret", "OracleCalls"]
metrics_list_radar = metrics_list + [metrics_list[0]]

fig = go.Figure()

for config_name, metrics in comparison_metrics.items():
    values = []
    for metric in metrics_list:
        val = metrics[metric]
        if metric in ["FinalError", "L1Error", "Regret"]:
            values.append(1.0 / (1.0 + val))
        else:
            values.append(val / 100)

    values_radar = values + [values[0]]

    fig.add_trace(
        go.Scatterpolar(
            r=values_radar, theta=metrics_list_radar, fill="toself", name=config_name
        )
    )

fig.update_layout(
    polar=dict(radialaxis=dict(visible=True, range=[0, 1])),
    title="Comprehensive Comparison Radar Chart",
    showlegend=True,
    height=600,
)

fig.show()

print("\n" + "=" * 80)
print("10. EXPORTING RESULTS")
print("=" * 80)

df_export = benchmark.get_dataframe()
csv_filename = "sthrd_benchmark_results.csv"
df_export.to_csv(csv_filename, index=False)
print(f"\nResults exported to: {csv_filename}")
print(f"File contains {len(df_export)} rows and {len(df_export.columns)} columns")

print("\nSummary statistics:")
print(df_export[["loss", "error", "x_norm", "theta_norm"]].describe())

print("\n" + "=" * 80)
print("ST-HRD BENCHMARK DEMONSTRATION COMPLETE!")
print("=" * 80)

ST-HRD BENCHMARK - COMPREHENSIVE DEMONSTRATION

1. BASIC SETUP - Simple SGD with Random Walk Drift


Running benchmark: 100%|██████████| 100/100 [00:00<00:00, 48669.11it/s]


Simple SGD Results:
Final tracking error: 0.0955
Metrics:
  NormalizedError: 0.0482
  L1Error: 0.2081
  DynamicRegret: 0.3766

Generating visualizations for Simple SGD...



2. COMPARING DIFFERENT OPTIMIZERS

Running SGD (lr=0.01)...
  Final error: 0.1148

Running SGD (lr=0.05)...
  Final error: 0.1463

Running Momentum SGD...
  Final error: 0.1005

Running Adam...
  Final error: 0.1325



3. TESTING DIFFERENT DRIFT TYPES

Testing Stationary drift...
  Final error: 0.0148

Testing Linear drift...
  Final error: 0.0961

Testing Random Walk drift...
  Final error: 0.1045

Testing Cyclic drift...
  Final error: 3.5773

Testing Teleportation drift...
  Final error: 0.0148


...

Running benchmark: 100%|██████████| 50/50 [00:00<00:00, 45275.30it/s]


Results:
Final error: 0.1693
Metrics:
  NormalizedError: 0.0490
  L1Error: 0.2090

DataFrame shape: (50, 12)

First few rows:
   time      loss  best_loss    x_norm  theta_norm     error       x_0  \
0     0  0.004967          0  0.001749    0.000000  0.001749 -0.000497   
1     1  0.110661          0  0.042430    0.050000  0.016234  0.008747   
2     2  0.064382          0  0.077808    0.063115  0.057579  0.008157   
3     3  0.319420          0  0.075626    0.057590  0.057617 -0.015968   
4     4  0.201605          0  0.120053    0.083567  0.072044 -0.054239   

    theta_0       x_1   theta_1       x_2   theta_2  
0  0.000000  0.000138  0.000000 -0.000648  0.000000  
1  0.014198 -0.007701 -0.003952  0.007383  0.018514  
2  0.008073 -0.008624  0.037364  0.038288  0.038591  
3 -0.000607  0.001446  0.028641  0.071001  0.043123  
4 -0.014229  0.012960  0.004104  0.081900  0.050736  



8. LEARNING RATE TUNING EXPERIMENT

Setting up learning rate tuning experiment...

Running learning rate sweep...
  lr=0.0010: avg error=0.0718 ± 0.0157
  lr=0.0050: avg error=0.1137 ± 0.0614
  lr=0.0100: avg error=0.0762 ± 0.0050
  lr=0.0200: avg error=0.1132 ± 0.0765
  lr=0.0500: avg error=0.1306 ± 0.0259
  lr=0.1000: avg error=0.1080 ± 0.0361
  lr=0.2000: avg error=0.1658 ± 0.0107



9. COMPREHENSIVE RADAR CHART COMPARISON

Testing SGD + Gaussian...
  Final error: 0.0627

Testing SGD + Pareto...
  Final error: 0.0660

Testing Adam + Linear Drift...
  Final error: 0.0356

Testing Momentum + Ravine...
  Final error: 0.0751



10. EXPORTING RESULTS

Results exported to: sthrd_benchmark_results.csv
File contains 50 rows and 12 columns

Summary statistics:
            loss      error     x_norm  theta_norm
count  50.000000  50.000000  50.000000   50.000000
mean    0.405968   0.118300   0.215258    0.204413
std     0.332356   0.045574   0.083616    0.068147
min     0.004967   0.001749   0.001749    0.000000
25%     0.168598   0.091435   0.158351    0.163356
50%     0.357212   0.117350   0.210415    0.227933
75%     0.505193   0.155875   0.288068    0.251275
max     1.539136   0.228645   0.352331    0.290873

ST-HRD BENCHMARK DEMONSTRATION COMPLETE!
